In [9]:
import re
import json
import six
from voluptuous import Schema, Required, All, Any, Range, Invalid, Length, ALLOW_EXTRA, MultipleInvalid

def validate_io_size(v):
    if re.search("^[0-9]+[bkm]", v) is None:
        raise Invalid("not a valid value (%s)" % str(v))

schema = Schema({
    "comment"                   : six.text_type,
    Required("operation")       : Any("read", "write"),
    "thread"                    : Range(min=1),
    Required("io_size")         : validate_io_size,
    Required("access_percentage")   : Range(min=1, max=100),
}, extra=ALLOW_EXTRA)

In [2]:
# valid data
schema({
    "comment"           : u"comment",
    "operation"         : "write",
    "thread"            : 8,
    "io_size"           : "8k",
    "access_percentage" : 100
})

{'access_percentage': 100,
 'comment': 'comment',
 'io_size': None,
 'operation': 'write',
 'thread': 8}

In [3]:
# valid data: omit some of the parameters
schema({
    "operation"         : "write",
    "io_size"           : "8k",
    "access_percentage" : 100
})

{'access_percentage': 100, 'io_size': None, 'operation': 'write'}

In [4]:
# invalid data: invalid "operation" value
try:
    schema({
        "comment"           : u"comment",
        "operation"         : "<invalid value>",
        "thread"            : 8,
        "io_size"           : "8k",
        "access_percentage" : 100
    })
except MultipleInvalid as e:
    six.print_(e)

not a valid value for dictionary value @ data['operation']


In [5]:
# invalid data: user defined invalidation
try:
    schema({
    "comment"           : u"comment",
    "operation"         : "write",
    "thread"            : 8,
    "io_size"           : "a8k",
    "access_percentage" : 100
    })
except MultipleInvalid as e:
    six.print_(e)

not a valid value (a8k) for dictionary value @ data['io_size']


In [21]:
schema = Schema(
    All([
        {six.text_type : Any(None, six.integer_types, float, six.text_type)},
    ], Length(min=1))
)

schema([
    {u'attr_b': '4', u'attr_c': u'a', u'attr_a': '1'},
    {u'attr_b': '2.1', u'attr_c': u'bb', u'attr_a': '2'},
    {u'attr_b': '120.9', u'attr_c': u'ccc', u'attr_a': '3'},
    {u'attr_b': None, u'attr_c': None}
])

try:
    schema([])
except MultipleInvalid as e:
    six.print_(e)

schema([{}])

try:
    schema([
        {u'attr_b': '4', u'attr_c': u'a', u'attr_a': {'nest': 1}},
    ])
except MultipleInvalid as e:
    six.print_(e)


length of value must be at least 1
not a valid value for dictionary value @ data[0]['attr_a']


In [24]:
sorted({u'attr_b': '4', u'attr_c': u'a', u'attr_a': '1'})

['attr_a', 'attr_b', 'attr_c']